## Import Libraries  & Utils

In [ ]:
import sys
sys.path.append('/paths_generator.py')
import paths_generator as pg
import random as rand
import math as m
import numpy as np
from scipy.optimize import dual_annealing

zones_num = pg.region.shape[0]*pg.region.shape[1]
print(zones_num)

81


## Opti problem

#### Constraint Function

In [ ]:
# It works just for one asset
def constraint(x): 
    
    constraints = []

    for asset in pg.assets:
      for road in pg.T_Ci[asset]:
        temp = []
        
        for i in range(len(road)-1):
          temp.append(x[road[i+1]][road[i]])

        constraints.append(1-sum(temp))   # All of them should be < 0
    
    return np.array(constraints)

#### Objective function (unconstrained problem)

In [ ]:
# Define the objective function
def objective(x, penalty=1e5): 
    print("dkhelt", x)
    S = 0
    x = np.array(x)
    x = np.reshape(x,(pg.zones_number, pg.zones_number))
    for i in range(zones_num):
        e_i = pg.region[pg.Rcount(i)] 
        for j in range(zones_num):
            S += x[i][j] * max(0, pg.slr - e_i)
    # Apply the constraint function and add penalties for violations
    constraints = constraint(x)
    if np.any(constraints > 0):
        S += penalty * np.sum(constraints)**2   # On applique le lagrangien!  # I don't need le min(max): it's defined explicitly
    
    print("9rib n returni!")
    return S 

## Dual Annealing

In [7]:
# Define the bounds for the decision variables
bounds = [(0, 1) for i in range(zones_num) for j in range(zones_num)]

# Call the Dual Annealing optimizer
result = dual_annealing(func=objective, bounds=bounds, maxiter=1000, seed=123)

# Print the optimal solution
print(result.x)
print(result.fun)

dkhelt [0.69646919 0.28613933 0.22685145 ... 0.05517737 0.78222669 0.01128399]
9rib n returni!
dkhelt [0.20368458 0.05527177 0.9959839  ... 0.56239277 0.28944208 0.78041643]
9rib n returni!
dkhelt [0.98985622 0.84144341 0.78215554 ... 0.34856441 0.30798466 0.798959  ]
9rib n returni!
dkhelt [0.77582208 0.62740927 0.14783567 ... 0.13453026 0.67366479 0.16463913]
9rib n returni!
dkhelt [0.82161519 0.37236425 0.19362878 ... 0.18032338 0.41861977 0.21043225]
9rib n returni!
dkhelt [0.18438928 0.16900128 0.55640288 ... 0.97696041 0.2152568  0.00706927]
9rib n returni!
dkhelt [0.62655274 0.61116474 0.07648034 ... 0.49703787 0.73533426 0.52714673]
9rib n returni!
dkhelt [0.88920911 0.64608417 0.3391367  ... 0.53195731 0.99799063 0.7898031 ]
9rib n returni!
dkhelt [0.01450312 0.77137819 0.46443072 ... 0.65725132 0.12328464 0.02566572]
9rib n returni!
dkhelt [0.53354922 0.29042429 0.98347682 ... 0.94860988 0.64233074 0.31702428]
9rib n returni!
dkhelt [0.69325887 0.80229464 0.49534717 ... 0.108